In [2]:
!pip install -q -r requirements.txt

In [5]:
from dotenv import load_dotenv
import os

from graph_db.db_connection import Neo4jConnection

load_dotenv()
uri = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

## General queries

In [15]:
conn = Neo4jConnection(uri, username, password)

query = """
MATCH (g:Genome)
RETURN COUNT(g) AS totalGenomes
"""

res = conn.query(query)
total_genomes = res[0].data()["totalGenomes"]
print("total genomes:", total_genomes)
      
query = """
MATCH (b:BGC)
RETURN COUNT(b) AS totalBGCs
"""

res = conn.query(query)
total_bgcs = res[0].data()["totalBGCs"]
print("total BGCs:", total_bgcs)

query = """
MATCH (p:Protein)
RETURN COUNT(p) AS totalProteins
"""

res = conn.query(query)
total_proteins =  res[0].data()["totalProteins"]
print("total Proteins:", total_proteins)



total genomes: 52325
total BGCs: 315634
total Proteins: 30076030


In [3]:
import pandas as pd


def classify_ocean_basin(lat, lon):
    if lat <= -60:
        return "Antarctic"
    elif lat >= 60 and lat <= 90:
        return "Arctic"
    elif -60 < lat < 60:
        if -80 <= lon <= 20:
            return "Atlantic"
        elif (20 < lon <= 180) or (-180 <= lon < -80):
            if -70 <= lon <= 120 and lat <= 30:
                return "Indian"
            else:
                return "Pacific"
    return "Unknown"


def records_to_dataframe(records):
    df = pd.DataFrame([record.data() for record in records])
    print(f"DataFrame columns: {df.columns}")
    print(f"DataFrame shape: {df.shape}")
    print(f"DataFrame head:\n{df.head()}")
    return df

In [4]:
import pandas as pd

def get_data_for_all_ocean_basins(conn, limit=None):
    limit_clause = f"LIMIT {limit}" if limit is not None else ""

    # 1. Taxonomic groups at phylum level
    phylum_query = f"""
    MATCH (g:Genome)-[:ORIGINATED_FROM]->(s:Sample)
    WHERE g.gtdb_classification IS NOT NULL
      AND s.latitude IS NOT NULL
      AND s.longigute IS NOT NULL
    RETURN 
       g.gtdb_classification AS gtdb_classification,
       toFloat(s.latitude) AS latitude,
       toFloat(s.longigute) AS longitude
    {limit_clause}
    """
    phylum_records = conn.query(phylum_query)
    phylum_df = records_to_dataframe(phylum_records)
    phylum_df['phylum'] = phylum_df['gtdb_classification'].apply(lambda x: x.split(';')[1] if x else None)
    phylum_df['ocean_basin'] = phylum_df.apply(lambda row: classify_ocean_basin(row['latitude'], row['longitude']), axis=1)
    phylum_data = phylum_df.groupby(['ocean_basin', 'phylum']).size().reset_index(name='count')

    # 2. Proteins per EC number (first digit)
    ec_query = f"""
    MATCH (p:Protein)
    MATCH (m:Genome)-[:CONTAINS]->(p)
    MATCH (m)-[:ORIGINATED_FROM]->(s:Sample)
    WHERE p.ec_numbers IS NOT NULL
      AND s.latitude IS NOT NULL
      AND s.longigute IS NOT NULL
    WITH p, s, rand() AS r
    ORDER BY r
    {limit_clause}
    RETURN 
       p.ec_numbers AS ec_numbers,
       toFloat(s.latitude) AS latitude,
       toFloat(s.longigute) AS longitude
    """
    ec_records = conn.query(ec_query)
    ec_df = records_to_dataframe(ec_records)
    ec_df = ec_df.explode('ec_numbers')
    ec_df['main_ec'] = ec_df['ec_numbers'].apply(lambda x: x.split('.')[0] if x else None)
    ec_df['ocean_basin'] = ec_df.apply(lambda row: classify_ocean_basin(row['latitude'], row['longitude']), axis=1)
    ec_data = ec_df.groupby(['ocean_basin', 'main_ec']).size().reset_index(name='count')

    # 3. BGCs per class
    bgc_query = f"""
    MATCH (bgc:BGC)
    MATCH (g:Genome)-[:CONTAINS]->(bgc)
    MATCH (g)-[:ORIGINATED_FROM]->(s:Sample)
    WHERE bgc.on_edge = "False"
      AND s.latitude IS NOT NULL
      AND s.longigute IS NOT NULL
    RETURN 
       bgc.bgc_class AS bgc_class,
       toFloat(s.latitude) AS latitude,
       toFloat(s.longigute) AS longitude
    {limit_clause}
    """
    bgc_records = conn.query(bgc_query)
    bgc_df = records_to_dataframe(bgc_records)
    bgc_df['ocean_basin'] = bgc_df.apply(lambda row: classify_ocean_basin(row['latitude'], row['longitude']), axis=1)
    bgc_data = bgc_df.groupby(['ocean_basin', 'bgc_class']).size().reset_index(name='count')

    return {
        'phylum_data': phylum_data,
        'ec_data': ec_data,
        'bgc_data': bgc_data
    }


try:
    conn = Neo4jConnection(uri, username, password)
    all_data = get_data_for_all_ocean_basins(conn, limit=50000)
    conn.close()
    print("Data retrieval successful")
    for key, df in all_data.items():
        print(f"\n{key} shape: {df.shape}")
        print(df.head())
except Exception as e:
    print(f"An error occurred: {str(e)}")
    import traceback
    traceback.print_exc()

DataFrame columns: Index(['gtdb_classification', 'latitude', 'longitude'], dtype='object')
DataFrame shape: (32325, 3)
DataFrame head:
                                 gtdb_classification  latitude  longitude
0  d__Bacteria;p__Proteobacteria;c__Gammaproteoba...   20.8322    63.6004
1  d__Bacteria;p__Proteobacteria;c__Gammaproteoba...   20.8322    63.6004
2  d__Bacteria;p__Proteobacteria;c__Gammaproteoba...   20.8322    63.6004
3  d__Bacteria;p__Proteobacteria;c__Gammaproteoba...   20.8322    63.6004
4  d__Bacteria;p__Proteobacteria;c__Gammaproteoba...   20.8322    63.6004


KeyboardInterrupt: 

## Query Proteins using a random sample

In [14]:
ec_query = f"""
MATCH (s:Sample)
WHERE s.latitude IS NOT NULL
  AND s.longigute IS NOT NULL
WITH s, rand() AS r
ORDER BY r
LIMIT 100000

MATCH (s)<-[:ORIGINATED_FROM]-(m:Genome)-[:CONTAINS]->(p:Protein)
WHERE p.ec_numbers IS NOT NULL
WITH s, collect(DISTINCT p.ec_numbers) AS ec_numbers
WHERE size(ec_numbers) > 0

RETURN 
   s.biosample_id AS sample_id,
   toFloat(s.latitude) AS latitude,
   toFloat(s.longigute) AS longitude,
   ec_numbers
"""

conn = Neo4jConnection(uri, username, password)
ec_records = conn.query(ec_query)

ec_df = records_to_dataframe(ec_records)
ec_df = ec_df.explode('ec_numbers')
ec_df['main_ec'] = ec_df['ec_numbers'].apply(lambda x: x.split('.')[0] if x else None)
ec_df['ocean_basin'] = ec_df.apply(lambda row: classify_ocean_basin(row['latitude'], row['longitude']), axis=1)
ec_data = ec_df.groupby(['ocean_basin', 'main_ec']).size().reset_index(name='count')

Query failed: {code: Neo.TransientError.General.DatabaseUnavailable} {message: The transaction has been terminated. Retry your operation in a new transaction, and you should see a successful result. The database is not currently available to serve your request, refer to the database logs for more details. Retrying your request at a later time may succeed. }


TypeError: 'NoneType' object is not iterable

In [13]:
ec_data.ocean_basin.unique()

array(['Arctic', 'Atlantic', 'Indian', 'Pacific'], dtype=object)

In [9]:
ec_data.to_csv("ec_data.tsv", sep="\t", index=False)

In [ ]:
import pandas as pd

df = pd.read_csv("outputs/phylum_data.tsv", sep="\t")
df.phylum = df.phylum.apply(lambda x:x.replace("p__", ""))
df.to_csv("outputs/phylum_data.tsv", sep="\t", index=False)

In [16]:
def save_data_to_tsv(data_dict, output_dir):
    """
    Save each DataFrame in the data dictionary to a TSV file in the specified directory.
    
    Args:
    data_dict (dict): Dictionary containing DataFrames to be saved
    output_dir (str): Path to the directory where TSV files will be saved
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    for key, df in data_dict.items():
        # Create a file name based on the key
        if key == "phylum_data":
            df.phylum = df.phylum.apply(lambda x:x.replace("p__", ""))
        file_name = f"{key}.tsv"
        file_path = os.path.join(output_dir, file_name)
        
        # Save the DataFrame to a TSV file
        df.to_csv(file_path, sep='\t', index=False)
        print(f"Saved {file_name} to {file_path}")

save_data_to_tsv(all_data, output_dir="outputs")


Saved phylum_data.tsv to outputs/phylum_data.tsv
Saved ec_data.tsv to outputs/ec_data.tsv
Saved bgc_data.tsv to outputs/bgc_data.tsv
